In [1]:
import datajoint as dj
import numpy as np

dj.config["display.limit"] = 20

In [2]:
schema = dj.schema("microns_ta3p100")

Connecting celiib@10.28.0.34:3306


In [10]:
ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")

In [9]:
@schema
class SynapseExcludeFinal(dj.Manual):
    definition="""
    -> ta3p100.Synapse
    """

In [16]:
#things need to include in new Synapse Exclude
"""
1) Those with ErrorLabel of SynapseCompartmentLabelFinal
2) The old SynapseExclude synapse which aren't form the earlier SynapseCompartLabel error labels migration

"""

#1) get the synapses with Error label for both the original proofreading session and the new proofreading session
synapse_ids_error_proof_original = (ta3p100.SynapseCompartmentLabel() & "segmentation=2" & "postsynaptic_label=10").fetch("synapse_id")
print(len(synapse_ids_error_proof_original))
synapse_ids_error_proof_new = (ta3p100.SynapseCompartmentLabelFinal() & "segmentation=2" & "postsynaptic_label=10").fetch("synapse_id")
print(len(synapse_ids_error_proof_new))

3724
10424


In [22]:
#inserting all the error from the proofreading stage 2 into new synapseExclude table
from tqdm import tqdm

total_dict = []
synapse_dicts = [dict(segmentation=2,synapse_id=sg) for sg in synapse_ids_error_proof_new]
len(synapse_dicts)
print(synapse_dicts[0])

#insert all into segment exclude
ta3p100.SynapseExcludeFinal().insert(synapse_dicts,skip_duplicates=True)

{'segmentation': 2, 'synapse_id': 742}


In [28]:
import collections

final_added_synapses = []

for q in tqdm(original_synapse_exclude):
    if q["synapse_id"] not in synapse_ids_error_proof_original:
        final_added_synapses.append(dict(segmentation=2,synapse_id=q["synapse_id"]))

100%|██████████| 72563/72563 [00:01<00:00, 59066.24it/s]


In [32]:
print(len(synapse_ids_error_proof_original)),print(len(final_added_synapses))
print(len(synapse_ids_error_proof_original) + len(final_added_synapses)),print(len(ta3p100.SynapseExclude()))

3724
68839
72563
72563


(None, None)

In [35]:
#add the last part of synapses that were not from proofreading to the synapseExclude
ta3p100.SynapseExcludeFinal.insert(final_added_synapses,skip_duplicates=True)